In [2]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetLMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
from torch.utils.data import DataLoader
import random

In [4]:
# Desactivar wandb completamente
os.environ["WANDB_DISABLED"] = "true"

In [5]:
print(f'available devices: {torch.cuda.device_count()}')
print(f'current device: { torch.cuda.current_device()}')

available devices: 1
current device: 0


In [6]:
# Verificar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [26]:
# Configuración de parámetros para generación de secuencias
model_name = "Rostlab/prot_xlnet"
batch_size = 8  # Reducido para generación que requiere más memoria
num_epochs = 5
learning_rate = 5e-5
max_length = 512
output_dir = "./prot_xlnet_generation_finetuned_cancer"

In [8]:
# Crear directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [9]:
# Cargar tokenizador y modelo para generación de lenguaje
print("Cargando tokenizador y modelo para generación de secuencias...")
tokenizer = XLNetTokenizer.from_pretrained(model_name)
# Usar XLNetLMHeadModel para generación de texto/secuencias
model = XLNetLMHeadModel.from_pretrained(model_name)

Cargando tokenizador y modelo para generación de secuencias...


c:\Users\alejandros\Documents\GitHub\Advanced_drug_discovery\.venv\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


In [10]:
print(f"Vocabulario del tokenizador: {tokenizer.vocab_size} tokens")
print("El modelo está configurado para generación de secuencias de proteínas")

Vocabulario del tokenizador: 37 tokens
El modelo está configurado para generación de secuencias de proteínas


In [11]:
# Mover el modelo al dispositivo adecuado
model.to(device)

XLNetLMHeadModel(
  (transformer): XLNetModel(
    (word_embedding): Embedding(37, 1024)
    (layer): ModuleList(
      (0-29): 30 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=1024, out_features=4096, bias=True)
          (layer_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): ReLU()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (lm_loss): Linear(in_features=1024, out_features=37, bias=True)
)

In [12]:
# Para congelar las capas base y solo entrenar las nuevas
for param in model.transformer.parameters():
    param.requires_grad = False

In [13]:
# Función para crear datos de ejemplo (reemplazar con secuencias reales)
def create_protein_sequences_data():
    """
    Crea un conjunto de datos de ejemplo de secuencias de proteínas.
    REEMPLAZA ESTA FUNCIÓN con la carga de tus secuencias reales.
    """
    # Secuencias de ejemplo más realistas
    sequences = [
        "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG",
        "MASNTVSAQGGSNRPVRDLASRQDFVRASSIIEKQLRDKVSADDLPVTLAQHLAVNFLHVLRLLE",
        "MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSSWRVVSSIEQKTEGAEKKQQ",
        "MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVEWNNPPSEKGLIVGHFSGIKYKGEKAQASEVDVNKMCCWVSKFKDAMRRYQGIQTCKIPGKVLSDLDAKIKAYNLTVEGVEGFVRYSRVTKQHVAAFLKELRHSKQYENVNLIHYILTDKRVDIQHLEKDLVKDLQAKGIQYLHQKGLKKQVPEGVKVP",
        "MGDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNLHGLFGRKTGQAPGYSYTAANKNKGIIWGEDTLMEYLENPKKYIPGTKMIFVGIKKKEERADLIAYLKKATNE"
    ]
    
    # Añadir más secuencias simuladas si necesitas más datos de prueba
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    for i in range(50):  # Reducido para este ejemplo
        seq_length = np.random.randint(80, 300)  # Longitudes más realistas
        seq = ''.join(np.random.choice(list(amino_acids)) for _ in range(seq_length))
        sequences.append(seq)
    
    return sequences

In [14]:
def load_real_protein_sequences(file_path=None):
    """
    Función para cargar tus secuencias reales de proteínas.
    Adapta esta función según el formato de tus datos.
    
    Formatos soportados:
    - CSV con columna 'sequence'
    - FASTA
    - Texto plano (una secuencia por línea)
    """
    if file_path is None:
        print("Usando datos de ejemplo. Para usar tus datos reales:")
        print("1. Modifica la función load_real_protein_sequences()")
        print("2. O pasa el path a tu archivo de secuencias")
        return create_protein_sequences_data()
    
    sequences = []
    
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
        if 'sequence' in df.columns:
            sequences = df['sequence'].tolist()
        else:
            print("El CSV debe tener una columna llamada 'sequence'")
            return create_protein_sequences_data()
    
    elif file_path.endswith('.fasta') or file_path.endswith('.fa'):
        with open(file_path, 'r') as f:
            current_seq = ""
            for line in f:
                if line.startswith('>'):
                    if current_seq:
                        sequences.append(current_seq)
                        current_seq = ""
                else:
                    current_seq += line.strip()
            if current_seq:
                sequences.append(current_seq)
    
    elif file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            sequences = [line.strip() for line in f if line.strip()]
    
    print(f"Cargadas {len(sequences)} secuencias de {file_path}")
    return sequences

In [15]:
# Cargar secuencias (modifica el path para usar tus datos reales)
#sequences = load_real_protein_sequences()  # Cambia por 
sequences = load_real_protein_sequences("../data/raw/peptides_cancer.csv")

print(f"Total de secuencias cargadas: {len(sequences)}")
print(f"Longitud promedio: {np.mean([len(seq) for seq in sequences]):.1f} aminoácidos")
print(f"Rango de longitudes: {min(len(seq) for seq in sequences)} - {max(len(seq) for seq in sequences)}")

Cargadas 6048 secuencias de ../data/raw/peptides_cancer.csv
Total de secuencias cargadas: 6048
Longitud promedio: 17.3 aminoácidos
Rango de longitudes: 2 - 97


In [17]:
# Dividir en conjuntos de entrenamiento y validación
train_sequences, val_sequences = train_test_split(sequences, test_size=0.2, random_state=42)

In [27]:
# Función para tokenizar las secuencias para generación
def tokenize_for_generation(examples):
    """
    Tokeniza las secuencias para entrenamiento de generación de lenguaje.
    Añade tokens especiales y prepara para masked language modeling.
    """
    # Tokenizar las secuencias
    tokenized = tokenizer(
        examples["sequence"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_special_tokens_mask=True
    )
    
    # Para XLNet, usamos los input_ids como labels también (language modeling)
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

In [19]:
# Crear datasets
train_dataset = Dataset.from_dict({"sequence": train_sequences})
val_dataset = Dataset.from_dict({"sequence": val_sequences})

In [20]:
# Tokenizar los datasets
train_dataset = train_dataset.map(tokenize_for_generation, batched=True)
val_dataset = val_dataset.map(tokenize_for_generation, batched=True)

Map:   0%|          | 0/4838 [00:00<?, ? examples/s]

Map:   0%|          | 0/1210 [00:00<?, ? examples/s]

In [21]:
# Configurar el data collator para language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # XLNet usa autoregressive LM, no masked LM
    mlm_probability=0.15
)

In [28]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=learning_rate,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=4,  # Aumentado para compensar batch size menor
    report_to=None,
    disable_tqdm=False,
    prediction_loss_only=True,  # Solo calculamos loss para generación
)

c:\Users\alejandros\Documents\GitHub\Advanced_drug_discovery\.venv\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [23]:
# Definir función de métricas para generación
def compute_generation_metrics(eval_pred):
    """
    Calcula métricas específicas para generación de secuencias.
    """
    predictions, labels = eval_pred
    # Para generación, nos enfocamos principalmente en la perplejidad (calculada del loss)
    return {}

In [29]:
# Inicializar el Trainer para generación
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_generation_metrics,
)

print("\nArquitectura del modelo:")
print(f"- Tipo: {type(model).__name__} (para generación de secuencias)")
print(f"- Parámetros: {sum(p.numel() for p in model.parameters()):,}")
print(f"- Vocabulario: {tokenizer.vocab_size} tokens")


Arquitectura del modelo:
- Tipo: XLNetLMHeadModel (para generación de secuencias)
- Parámetros: 409,413,669
- Vocabulario: 37 tokens


In [31]:
# Entrenar el modelo
print("\nIniciando entrenamiento para generación de secuencias...")
print("Este modelo aprenderá a generar secuencias de proteínas similares a las del entrenamiento")
trainer.train()


Iniciando entrenamiento para generación de secuencias...
Este modelo aprenderá a generar secuencias de proteínas similares a las del entrenamiento


  0%|          | 0/755 [00:00<?, ?it/s]

{'loss': 12.6694, 'grad_norm': 2.5340099334716797, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.07}
{'loss': 12.7472, 'grad_norm': 2.5389554500579834, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.13}
{'loss': 12.6117, 'grad_norm': 2.534273862838745, 'learning_rate': 3e-06, 'epoch': 0.2}
{'loss': 12.744, 'grad_norm': 2.530470848083496, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.26}
{'loss': 12.798, 'grad_norm': 2.5221543312072754, 'learning_rate': 5e-06, 'epoch': 0.33}
{'loss': 12.7354, 'grad_norm': 2.526870012283325, 'learning_rate': 6e-06, 'epoch': 0.4}
{'loss': 12.6937, 'grad_norm': 2.5525267124176025, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.46}
{'loss': 12.7087, 'grad_norm': 2.5521256923675537, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.53}
{'loss': 12.8252, 'grad_norm': 2.52350115776062, 'learning_rate': 9e-06, 'epoch': 0.6}
{'loss': 12.795, 'grad_norm': 2.5364279747009277, 'learning_rate': 1e-05, 'epoch': 0.66}
{'loss': 12.7503, 'grad_norm'

  0%|          | 0/152 [00:00<?, ?it/s]

{'eval_loss': 4.14453125, 'eval_runtime': 12.7622, 'eval_samples_per_second': 94.811, 'eval_steps_per_second': 11.91, 'epoch': 1.0}
{'loss': 12.7814, 'grad_norm': 2.542156934738159, 'learning_rate': 1.6000000000000003e-05, 'epoch': 1.06}
{'loss': 12.7792, 'grad_norm': 2.521373987197876, 'learning_rate': 1.7000000000000003e-05, 'epoch': 1.12}
{'loss': 12.8394, 'grad_norm': 2.530184507369995, 'learning_rate': 1.8e-05, 'epoch': 1.19}
{'loss': 12.5987, 'grad_norm': 2.516324996948242, 'learning_rate': 1.9e-05, 'epoch': 1.26}
{'loss': 12.8314, 'grad_norm': 2.555100440979004, 'learning_rate': 2e-05, 'epoch': 1.32}
{'loss': 12.6418, 'grad_norm': 2.4924168586730957, 'learning_rate': 2.1e-05, 'epoch': 1.39}
{'loss': 12.6099, 'grad_norm': 2.5245957374572754, 'learning_rate': 2.2000000000000003e-05, 'epoch': 1.45}
{'loss': 12.7792, 'grad_norm': 2.5380361080169678, 'learning_rate': 2.3000000000000003e-05, 'epoch': 1.52}
{'loss': 12.8683, 'grad_norm': 2.522397518157959, 'learning_rate': 2.4e-05, 'ep

...

  0%|          | 0/152 [00:00<?, ?it/s]

{'eval_loss': 4.1194658279418945, 'eval_runtime': 12.8731, 'eval_samples_per_second': 93.994, 'eval_steps_per_second': 11.808, 'epoch': 4.99}


There were missing keys in the checkpoint model loaded: ['lm_loss.weight'].


{'train_runtime': 362.0036, 'train_samples_per_second': 66.823, 'train_steps_per_second': 2.086, 'train_loss': 12.700960424562163, 'epoch': 4.99}


TrainOutput(global_step=755, training_loss=12.700960424562163, metrics={'train_runtime': 362.0036, 'train_samples_per_second': 66.823, 'train_steps_per_second': 2.086, 'total_flos': 3.0373613146251264e+16, 'train_loss': 12.700960424562163, 'epoch': 4.991735537190083})

In [32]:
# Guardar el modelo y el tokenizador
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Modelo de generación guardado en {output_dir}")

Modelo de generación guardado en ./prot_xlnet_generation_finetuned_cancer


In [33]:
# Evaluar el modelo
print("Evaluando el modelo...")
eval_results = trainer.evaluate()
print(f"Resultados de evaluación: {eval_results}")

Evaluando el modelo...


  0%|          | 0/152 [00:00<?, ?it/s]

Resultados de evaluación: {'eval_loss': 4.1194658279418945, 'eval_runtime': 12.7203, 'eval_samples_per_second': 95.124, 'eval_steps_per_second': 11.949, 'epoch': 4.991735537190083}


In [59]:
# Funciones para generar nuevas secuencias
def generate_protein_sequence(prompt="", max_new_tokens=100, temperature=0.8, do_sample=True, repetition_penalty = 1.1):
    """
    Genera una nueva secuencia de proteína.
    
    Args:
        prompt: Secuencia inicial (puede estar vacía)
        max_new_tokens: Número máximo de tokens nuevos a generar
        temperature: Controla la aleatoriedad (más alto = más aleatorio)
        do_sample: Si usar sampling o greedy decoding
    """
    model.eval()
    
    # Si no hay prompt, usar un token especial o secuencia corta común
    if not prompt:
        prompt = "M"  # Muchas proteínas empiezan con metionina
    
    # Tokenizar el prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        # Generar secuencia
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty= repetition_penalty #1.1
        )
    
    # Decodificar la secuencia generada
    generated_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remover el prompt original para obtener solo la parte generada
    if prompt:
        generated_sequence = generated_sequence[len(prompt):]
    
    return generated_sequence


In [36]:
def generate_multiple_sequences(num_sequences=5, **generation_kwargs):
    """
    Genera múltiples secuencias de proteínas.
    """
    sequences = []
    for i in range(num_sequences):
        seq = generate_protein_sequence(**generation_kwargs)
        sequences.append(seq)
        print(f"Secuencia {i+1}: {seq}")
    return sequences

In [61]:
# Ejemplos de generación
print("\n" + "="*50)
print("EJEMPLOS DE GENERACIÓN DE SECUENCIAS")
print("="*50)

# Generar secuencias sin prompt
print("\n1. Generación sin prompt inicial:")
generated_seqs = generate_multiple_sequences(
    num_sequences=20,
    max_new_tokens=17,
    temperature=3.0,
    repetition_penalty = 5.0

)


EJEMPLOS DE GENERACIÓN DE SECUENCIAS

1. Generación sin prompt inicial:
Secuencia 1: ()) W H–" S
Secuencia 2:  Q P€ F-– V£.
Secuencia 3: € T"
Secuencia 4:  A A G K Y N" TX K L K A D R V Y
Secuencia 5: - D
Secuencia 6:  V"( R. S€ G T Q) T£
Secuencia 7:  Q
Secuencia 8:  N T D C"£ A P( A–
Secuencia 9: £€-)£. G(–
Secuencia 10: . G D Y T-
Secuencia 11: 
Secuencia 12:  W."€ C- N£. P–€
Secuencia 13: 
Secuencia 14:  A G€ R(()€ P– F" T R
Secuencia 15: X A K
Secuencia 16: -( Y£" S
Secuencia 17: .("–)..€
Secuencia 18:  T G–".(££ A€
Secuencia 19: € S D-X G T
Secuencia 20: .


In [62]:
# Generar con diferentes temperaturas
print("\n2. Generación con diferentes temperaturas:")
for temp in [1.0, 1.2, 1.5, 2.0, 2.5, 3.0]:
    print(f"\nTemperatura {temp}:")
    seq = generate_protein_sequence(
        prompt="AAVALLPAVLLALLAPQLGKKKHRRRPSKKKRHW",
        max_new_tokens=60,
        temperature=temp,
         repetition_penalty = 5.0

    )
    print(f"  {seq}")


2. Generación con diferentes temperaturas:

Temperatura 1.0:
   N I V S Q A A A I R V N S V I S Q T P N T R I N S N I S R V L F Q M Q Y K G Q Q N P

Temperatura 1.2:
  

Temperatura 1.5:
  

Temperatura 2.0:
  

Temperatura 2.5:
  E I L R M M I F F S V Y L R T N H P S V T D G W W M W I K E I G T A T S M A S R G F

Temperatura 3.0:
  P L V P F E P T I P Q R L Y H A S A F V P E M R N E A S M A R T M A F S D S G P K I E S


In [65]:
# Generar con prompt específico
print("\n3. Generación con prompt específico:")
prompt = 'AAVALLPAVLLALLAPQLGKKKHRRRPSKKKRHW'
extended_seq = generate_protein_sequence(
    prompt=prompt,
    max_new_tokens=100,
    temperature=2.0,
    repetition_penalty = 5.0
    )
print(f"Prompt: {prompt}")
print(f"Generado: {extended_seq}")



3. Generación con prompt específico:
Prompt: AAVALLPAVLLALLAPQLGKKKHRRRPSKKKRHW
Generado: C R-


In [68]:
generated_seqs2 = generate_multiple_sequences(
    num_sequences=20,
    max_new_tokens=50,
    temperature=2.0,
    prompt = 'A A V A L L P A V L L A L L A P Q L G K K K H R R R P S K K K R H W',
    repetition_penalty = 5.0

)

Secuencia 1:  E D D Y S E S W Y H S A S M N V D G V L S V P P S Q P K R K C Y N N H Y W R Q K S I F K I G N I V Q
Secuencia 2:  N K S P S C P D Q D T L Q P P I K M A M E L S E H H K L T I N S Y K F Q E T R H F I V T Q S K S N G
Secuencia 3:  P T I F T T Y W I F R Y F V Y F W F P F I P T C V I Y H Y S L V R G H K C A A W K Y V C A T V W C W
Secuencia 4:  K N N R C N R R M N N V G W P C S C T V I V L V V A E Y F L I T T I Q E S M R F W S V R C V V G M T
Secuencia 5:  D L E Q P F E A Y Q M M R G R G S G F A Y N Y L C S M T N F G P V T N Y Q V V V S G G N W M K R W M
Secuencia 6:  S W D DX N I T K Y P H K Q L E Y W G N T W D V P R S N S E L N S L M D W A L V I D C Q Q M I V I E
Secuencia 7:  N C P E A G K Y C Q S D S C F I Q T M M D F G S N T A E D C A T F Q D D A F F F G G P G S S L R K D
Secuencia 8:  S K N C H P H L G RX W Q K M Q G E Q W T L A L M D T T T P K L M V R L S L T F W E R V K G D S G C
Secuencia 9:  L F V W W W G P R A K V C N L V D V K L G S F Y N K A F F I K G I Y V L M V 